In [15]:
from langchain.document_loaders import TextLoader
sota_loader = TextLoader("../../modules/state_of_the_union.txt")
pg_loader = TextLoader("../../../../gpt_index/examples/paul_graham_essay/data/paul_graham_essay.txt")

In [12]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import FAISS

In [13]:
sota_index = VectorstoreIndexCreator(vectorstore_cls=FAISS).from_loaders([sota_loader])


In [16]:
pg_index = VectorstoreIndexCreator(vectorstore_kwargs={"collection_name": "paul-graham"}).from_loaders([pg_loader])


Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [17]:
sota_index.query("what did the president about kentaji brown jackson?")

" The President nominated Circuit Court of Appeals Judge Ketanji Brown Jackson to serve on the United States Supreme Court. He said she is one of the nation's top legal minds and will continue Justice Breyer's legacy of excellence."

In [18]:
pg_index.query("what did the president about kentaji brown jackson?")

" Kentaji Brown Jackson was not mentioned in the context, so I don't know."

In [19]:
from langchain.agents import initialize_agent, Tool
from langchain.tools import BaseTool
from langchain.llms import OpenAI

In [20]:
tools = [
    Tool(
        name = "State of Union QA System",
        func=sota_index.query,
        description="useful for when you need to answer questions about the most recent state of the union address. Input should be a fully formed question."
    ),
    Tool(
        name = "Paul Graham QA System",
        func=pg_index.query,
        description="useful for when you need to answer questions about Paul Graham. Input should be a fully formed question."
    ),
]

In [22]:
agent = initialize_agent(tools, OpenAI(temperature=0), agent="zero-shot-react-description", verbose=True)

In [24]:
import json

In [25]:
with open("../../../notebooks/state_of_union_qa.json") as f:
    sota_qa = json.load(f)

In [26]:
with open("../../../notebooks/paul_graham_qa.json") as f:
    pg_qa = json.load(f)

In [28]:
for d in sota_qa:
    d['steps'] = [{"tool": "State of Union QA System"}, {"tool_input": d["question"]}]
for d in pg_qa:
    d['steps'] = [{"tool": "Paul Graham QA System"}, {"tool_input": d["question"]}]

In [30]:
all_vectorstore_routing = sota_qa + pg_qa

In [31]:
with open("vectorstore_sota_pg.json", "w") as f:
    json.dump(all_vectorstore_routing, f)

In [33]:
all_vectorstore_routing = [{'question': 'What is the purpose of the NATO Alliance?',
  'answer': 'The purpose of the NATO Alliance is to secure peace and stability in Europe after World War 2.',
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': 'What is the purpose of the NATO Alliance?'}]},
 {'question': 'What is the U.S. Department of Justice doing to combat the crimes of Russian oligarchs?',
  'answer': 'The U.S. Department of Justice is assembling a dedicated task force to go after the crimes of Russian oligarchs.',
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': 'What is the U.S. Department of Justice doing to combat the crimes of Russian oligarchs?'}]},
 {'question': 'What is the American Rescue Plan and how did it help Americans?',
  'answer': 'The American Rescue Plan is a piece of legislation that provided immediate economic relief for tens of millions of Americans. It helped put food on their table, keep a roof over their heads, and cut the cost of health insurance. It created jobs and left no one behind.',
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': 'What is the American Rescue Plan and how did it help Americans?'}]},
 {'question': 'What is the purpose of the Bipartisan Innovation Act mentioned in the text?',
  'answer': 'The Bipartisan Innovation Act will make record investments in emerging technologies and American manufacturing to level the playing field with China and other competitors.',
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': 'What is the purpose of the Bipartisan Innovation Act mentioned in the text?'}]},
 {'question': "What is Joe Biden's plan to fight inflation?",
  'answer': "Joe Biden's plan to fight inflation is to lower costs, not wages, by making more goods in America, increasing the productive capacity of the economy, and cutting the cost of prescription drugs, energy, and child care.",
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': "What is Joe Biden's plan to fight inflation?"}]},
 {'question': 'What is the proposed minimum tax rate for corporations under the plan?',
  'answer': 'The proposed minimum tax rate for corporations is 15%.',
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': 'What is the proposed minimum tax rate for corporations under the plan?'}]},
 {'question': 'What are the four common sense steps that the author suggests to move forward safely?',
  'answer': 'The four common sense steps suggested by the author to move forward safely are: stay protected with vaccines and treatments, prepare for new variants, end the shutdown of schools and businesses, and stay vigilant.',
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': 'What are the four common sense steps that the author suggests to move forward safely?'}]},
 {'question': 'What is the purpose of the American Rescue Plan?',
  'answer': 'The purpose of the American Rescue Plan is to provide $350 Billion that cities, states, and counties can use to hire more police and invest in proven strategies like community violence interruption.',
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': 'What is the purpose of the American Rescue Plan?'}]},
 {'question': 'What measures does the speaker ask Congress to pass to reduce gun violence?',
  'answer': 'The speaker asks Congress to pass universal background checks, ban assault weapons and high-capacity magazines, and repeal the liability shield that makes gun manufacturers the only industry in America that can’t be sued.',
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': 'What measures does the speaker ask Congress to pass to reduce gun violence?'}]},
 {'question': 'What is the Unity Agenda for the Nation that the President is offering?',
  'answer': 'The Unity Agenda for the Nation includes four big things that can be done together: beat the opioid epidemic, take on mental health, support veterans, and strengthen the Violence Against Women Act.',
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': 'What is the Unity Agenda for the Nation that the President is offering?'}]},
 {'question': 'What is the purpose of ARPA-H?',
  'answer': 'ARPA-H will have a singular purpose—to drive breakthroughs in cancer, Alzheimer’s, diabetes, and more.',
  'steps': [{'tool': 'State of Union QA System'},
   {'tool_input': 'What is the purpose of ARPA-H?'}]},
 {'question': 'What were the two main things the author worked on before college?',
  'answer': 'The two main things the author worked on before college were writing and programming.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What were the two main things the author worked on before college?'}]},
 {'question': 'What made the author want to work on AI?',
  'answer': "The novel 'The Moon is a Harsh Mistress' and a PBS documentary showing Terry Winograd using SHRDLU made the author want to work on AI.",
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What made the author want to work on AI?'}]},
 {'question': 'What did the author realize while looking at a painting at the Carnegie Institute?',
  'answer': 'The author realized that paintings were something that could be made to last and that making them was a way to be independent and make a living.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What did the author realize while looking at a painting at the Carnegie Institute?'}]},
 {'question': 'What did the author write their dissertation on?',
  'answer': 'The author wrote their dissertation on applications of continuations.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What did the author write their dissertation on?'}]},
 {'question': 'What is the difference between painting still lives and painting people?',
  'answer': "Painting still lives is different from painting people because the subject, as its name suggests, can't move. People can't sit for more than about 15 minutes at a time, and when they do they don't sit very still. So the traditional m.o. for painting people is to know how to paint a generic person, which you then modify to match the specific person you're painting.",
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What is the difference between painting still lives and painting people?'}]},
 {'question': 'What did the author learn while working at Interleaf?',
  'answer': "The author learned that low end software tends to eat high end software, that it's better for technology companies to be run by product people than sales people, that it leads to bugs when code is edited by too many people, that cheap office space is no bargain if it's depressing, that planned meetings are inferior to corridor conversations, that big, bureaucratic customers are a dangerous source of money, and that there's not much overlap between conventional office hours and the optimal time for hacking, or conventional offices and the optimal place for it.",
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What did the author learn while working at Interleaf?'}]},
 {'question': 'What did the author do to survive during the next several years after leaving RISD?',
  'answer': 'The author did freelance work for the group that did projects for customers to survive for the next several years after leaving RISD.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What did the author do to survive during the next several years after leaving RISD?'}]},
 {'question': "What was the author's motivation for wanting to become rich?",
  'answer': 'The author wanted to become rich so that he could work on whatever he wanted.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': "What was the author's motivation for wanting to become rich?"}]},
 {'question': 'What is Viaweb and how did it get its name?',
  'answer': 'Viaweb is a company that built a web app for creating online stores. It got its name from the fact that the software worked via the web.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What is Viaweb and how did it get its name?'}]},
 {'question': 'What was the price charged by Viaweb for a small store and a big one?',
  'answer': '$100 a month for a small store and $300 a month for a big one.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What was the price charged by Viaweb for a small store and a big one?'}]},
 {'question': 'Why did the author hire more people for his startup?',
  'answer': "The author hired more people for his startup partly because the investors wanted him to and partly because that's what startups did during the Internet Bubble.",
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'Why did the author hire more people for his startup?'}]},
 {'question': "What was the author's idea for a new company?",
  'answer': "The author's idea was to build a web app for making web apps, where people could edit code on their server through the browser and then host the resulting applications for them.",
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': "What was the author's idea for a new company?"}]},
 {'question': "What was the author's turning point in figuring out what to work on?",
  'answer': "The author's turning point in figuring out what to work on was when he started publishing essays online.",
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': "What was the author's turning point in figuring out what to work on?"}]},
 {'question': 'What is the danger for the ambitious according to the text?',
  'answer': 'The desire to impress people is the danger for the ambitious according to the text.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What is the danger for the ambitious according to the text?'}]},
 {'question': 'What is the most distinctive thing about Y Combinator?',
  'answer': 'The most distinctive thing about YC is the batch model: to fund a bunch of startups all at once, twice a year, and then to spend three months focusing intensively on trying to help them.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What is the most distinctive thing about Y Combinator?'}]},
 {'question': 'What was the Summer Founders Program and how many groups were selected for funding?',
  'answer': 'The Summer Founders Program was a program for undergrads to apply for funding for their startup ideas. 8 groups were selected for funding out of 225 applications.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What was the Summer Founders Program and how many groups were selected for funding?'}]},
 {'question': 'What was the biggest source of stress for the author while working at YC?',
  'answer': 'HN (Hacker News)',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What was the biggest source of stress for the author while working at YC?'}]},
 {'question': 'What did the author decide to do after leaving YC?',
  'answer': 'The author decided to focus on painting.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What did the author decide to do after leaving YC?'}]},
 {'question': 'What is the distinctive thing about Lisp?',
  'answer': 'The distinctive thing about Lisp is that its core is a language defined by writing an interpreter in itself.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What is the distinctive thing about Lisp?'}]},
 {'question': 'Why did the author move to England?',
  'answer': 'The author moved to England to let their kids experience living in another country and because the author was a British citizen by birth.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'Why did the author move to England?'}]},
 {'question': 'What was the reason behind the change of name from Cambridge Seed to Y Combinator?',
  'answer': "They didn't want a regional name, in case someone copied them in Silicon Valley, so they renamed themselves after one of the coolest tricks in the lambda calculus, the Y combinator.",
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What was the reason behind the change of name from Cambridge Seed to Y Combinator?'}]},
 {'question': 'What is the purpose of YC?',
  'answer': 'The purpose of YC is to cause startups to be founded that would not otherwise have existed.',
  'steps': [{'tool': 'Paul Graham QA System'},
   {'tool_input': 'What is the purpose of YC?'}]}]